### Feature Store Creation

In [ ]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col


connection_parameters = {
    "account": "<your snowflake account identifier>",
    "user": "<your snowflake username>",
    "password": "<your snowflake password>",
    "role": "<your snowflake role>",
    "warehouse": "<your snowflake warehouse>",  # optional
    "database": "<your snowflake database>",  # optional
    "schema": "<your snowflake schema>" # optional
  }

session = Session.builder.configs(connection_parameters).create()

In [ ]:
#CREATE FEATURE STORE

from snowflake.ml.feature_store import (FeatureStore, FeatureView, Entity, CreationMode)

fs = FeatureStore(
    session=session,
    database="SNOWPARK",
    name="TUTORIAL",
    default_warehouse="COMPUTE_WH",
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST,
)

### Creating & Registering Feature Entity

In [ ]:
entity = Entity(name="WEATHER", join_keys=["ID"])
fs.register_entity(entity)
fs.list_entities().show()

----------------------------------
|"NAME"   |"JOIN_KEYS"  |"DESC"  |
----------------------------------
|WEATHER  |ID           |        |
----------------------------------



In [ ]:
df = session.table("model_data")
df.show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SEASON"  |"HOLIDAY"  |"WORKINGDAY"  |"WEATHER"  |"TEMP"  |"ATEMP"  |"HUMIDITY"  |"WINDSPEED"       |"CASUAL"  |"REGISTERED"  |"COUNT"  |"DATETIME"           |"HOUR"  |"MONTH"  |"DATE"      |"WEEKDAY"  |"YEAR"  |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1         |0          |0             |1          |9.84    |14.395   |81          |12.7993954069447  |3         |13            |16       |2011-01-01 00:00:00  |0       |1        |2011-01-01  |6          |2011    |
|1         |0          |0             |1          |9.02    |13.635   |80          |12.7993954069447  |8         |32            |40       |2011-0

### Creating & Registering Feature Views

In [ ]:
from snowflake.snowpark.types import IntegerType

# CREATING ID COLUMN
from snowflake.snowpark.functions import monotonically_increasing_id
df = df.withColumn("ID", monotonically_increasing_id())

#CREATING A NEW FEATURE
df = df.withColumn("QUARTER", ((df["MONTH"] + 2) / 3).cast(IntegerType()))
df.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SEASON"  |"HOLIDAY"  |"WORKINGDAY"  |"WEATHER"  |"TEMP"  |"ATEMP"  |"HUMIDITY"  |"WINDSPEED"       |"CASUAL"  |"REGISTERED"  |"COUNT"  |"DATETIME"           |"HOUR"  |"MONTH"  |"DATE"      |"WEEKDAY"  |"YEAR"  |"ID"  |"QUARTER"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1         |0          |0             |1          |9.84    |14.395   |81          |12.7993954069447  |3         |13            |16       |2011-01-01 00:00:00  |0       |1        |2011-01-01  |6          |2011    |0     |1          |
|1         |0          |0             |1          |9.02    |13.635  

In [ ]:
# SELECTING COLUMNS FOR FIRST FEATURE VIEW
feature_df = df.select(["SEASON", "WEATHER", "TEMP", "ATEMP", "HUMIDITY", "WINDSPEED", "QUARTER", "YEAR", "ID"])

feature_df.show()

-------------------------------------------------------------------------------------------------------
|"SEASON"  |"WEATHER"  |"TEMP"  |"ATEMP"  |"HUMIDITY"  |"WINDSPEED"       |"QUARTER"  |"YEAR"  |"ID"  |
-------------------------------------------------------------------------------------------------------
|1         |1          |9.84    |14.395   |81          |12.7993954069447  |1          |2011    |0     |
|1         |1          |9.02    |13.635   |80          |12.7993954069447  |1          |2011    |1     |
|1         |1          |9.02    |13.635   |80          |12.7993954069447  |1          |2011    |2     |
|1         |1          |9.84    |14.395   |75          |12.7993954069447  |1          |2011    |3     |
|1         |1          |9.84    |14.395   |75          |12.7993954069447  |1          |2011    |4     |
|1         |2          |9.84    |12.88    |75          |6.0032            |1          |2011    |5     |
|1         |1          |9.02    |13.635   |80          |12.79939

In [ ]:
#CREATING FEATURE VIEW

fv = FeatureView(
    name="WEATHER_FEATURES",
    entities=[entity],
    feature_df=feature_df,
    desc="weather features"
)

fv = fs.register_feature_view(
    feature_view=fv,
    version="V1",
    refresh_freq="1 minute",
    block=True
)

In [ ]:
# EXAMINE FEATURE VIEW CONTENT

fs.read_feature_view(fv).show()

FeatureStore.read_feature_view() is in private preview since 1.0.8. Do not use it in production. 


-------------------------------------------------------------------------------------------------------
|"SEASON"  |"WEATHER"  |"TEMP"  |"ATEMP"  |"HUMIDITY"  |"WINDSPEED"       |"QUARTER"  |"YEAR"  |"ID"  |
-------------------------------------------------------------------------------------------------------
|1         |1          |9.84    |14.395   |81          |12.7993954069447  |1          |2011    |0     |
|1         |1          |9.02    |13.635   |80          |12.7993954069447  |1          |2011    |1     |
|1         |1          |9.02    |13.635   |80          |12.7993954069447  |1          |2011    |2     |
|1         |1          |9.84    |14.395   |75          |12.7993954069447  |1          |2011    |3     |
|1         |1          |9.84    |14.395   |75          |12.7993954069447  |1          |2011    |4     |
|1         |2          |9.84    |12.88    |75          |6.0032            |1          |2011    |5     |
|1         |1          |9.02    |13.635   |80          |12.79939

In [ ]:
#CREATE RENTAL FEATURES

rental_df = df.select(["HOLIDAY", "WORKINGDAY", "CASUAL", "REGISTERED", "COUNT", "HOUR", "MONTH", "WEEKDAY", "ID"])

rental_fv = FeatureView(
    name="RENTAL_FEATURES",
    entities=[entity],
    feature_df=rental_df,
    desc="rental features"
)
rental_fv = fs.register_feature_view(
    feature_view=rental_fv,
    version="V1",
    refresh_freq="1 minute",
    block=True
)

/Users/paranthamansrinivasan/Environment/piperr/lib/python3.10/site-packages/snowflake/ml/feature_store/feature_store.py:1098: UserWarning: Dynamic table: `SNOWPARK.TUTORIAL.RENTAL_FEATURES$V1` will not refresh in INCREMENTAL mode. It will likely incurr bigger computation cost. The reason is: Query contains the function 'SEQ8', but change tracking is not supported on queries with non-deterministic functions.
  warnings.warn(


In [ ]:
# LIST ALL FEATURE VIEW
fs.list_feature_views(entity_name="WEATHER").select(["NAME", "ENTITIES", "FEATURE_DESC"]).show()

FeatureStore.list_feature_views() is in private preview since 1.0.8. Do not use it in production. 


--------------------------------------------------------------------
|"NAME"            |"ENTITIES"             |"FEATURE_DESC"         |
--------------------------------------------------------------------
|RENTAL_FEATURES   |[                      |{                      |
|                  |  {                    |  "CASUAL": null,      |
|                  |    "desc": "",        |  "COUNT": null,       |
|                  |    "join_keys": [     |  "HOLIDAY": null,     |
|                  |      "ID"             |  "HOUR": null,        |
|                  |    ],                 |  "MONTH": null,       |
|                  |    "name": "WEATHER"  |  "REGISTERED": null,  |
|                  |  }                    |  "WEEKDAY": null,     |
|                  |]                      |  "WORKINGDAY": null   |
|                  |                       |}                      |
|WEATHER_FEATURES  |[                      |{                      |
|                  |  {           

In [ ]:
# COMBINING TWO FEATURES

full_fv = fs.merge_features(features=[fv, rental_fv], name="FULL_FEATURES")
full_fv = fs.register_feature_view(feature_view=full_fv, version="V1")

FeatureStore.merge_features() is in private preview since 1.0.8. Do not use it in production. 


### Data Generation through Feature views

In [ ]:
#GENERATING TRAINING DATA
spine_df = session.table("MODEL_DATA")
spine_df = spine_df.withColumn("ID", monotonically_increasing_id())
spine_df = spine_df.select("ID", "COUNT")
spine_df.show()


training_data = fs.generate_dataset(
    spine_df=spine_df,
    features=[
        full_fv.slice([
            "CASUAL","HOLIDAY","HOUR","MONTH","REGISTERED","WEEKDAY","WORKINGDAY","ATEMP",
            "HUMIDITY","QUARTER","SEASON","TEMP","WEATHER","WINDSPEED","YEAR"
        ])
    ],
    materialized_table="TRAINING_DATA",
    spine_timestamp_col=None,
    spine_label_cols=["COUNT"],
    save_mode="merge",
    exclude_columns=['ID']
)

training_data.df.show()

FeatureStore.generate_dataset() is in private preview since 1.0.8. Do not use it in production. 


------------------
|"ID"  |"COUNT"  |
------------------
|0     |16       |
|1     |40       |
|2     |32       |
|3     |13       |
|4     |1        |
|5     |1        |
|6     |2        |
|7     |3        |
|8     |8        |
|9     |14       |
------------------

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"COUNT"  |"CASUAL"  |"HOLIDAY"  |"HOUR"  |"MONTH"  |"REGISTERED"  |"WEEKDAY"  |"WORKINGDAY"  |"ATEMP"  |"HUMIDITY"  |"QUARTER"  |"SEASON"  |"TEMP"  |"WEATHER"  |"WINDSPEED"       |"YEAR"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|16       |3         |0          |0       |1        |13            |6          |0             |14.395   |81          |1          |1         |9.84    |1         

### Model Training & Prediction Using Enriched Data

In [ ]:
# TRAIN A MODEL

from snowflake.ml.modeling.model_selection import GridSearchCV
from snowflake.ml.modeling.ensemble import GradientBoostingRegressor

FEATURE_LIST = [ "HOLIDAY", "WORKINGDAY", "WEEKDAY", "HOUR", "HUMIDITY", "MONTH", "TEMP", "YEAR", "ATEMP", "WINDSPEED", "SEASON", "WEATHER"]
LABEL_COLUMNS = ['COUNT']
OUTPUT_COLUMNS = ['PREDICTED_COUNT']

param_grid = {
        "n_estimators":[100, 200, 300, 400, 500],
        "learning_rate":[0.1, 0.2, 0.3, 0.4, 0.5],
}

grid_search = GridSearchCV(
    estimator=GradientBoostingRegressor(),
    param_grid=param_grid,
    n_jobs = -1,
    scoring="neg_root_mean_squared_error",
    input_cols=FEATURE_LIST,
    label_cols=LABEL_COLUMNS,
    output_cols=OUTPUT_COLUMNS
)

train_df = training_data.df.drop(["ID"])

In [ ]:
grid_search.fit(train_df)

In [ ]:
#PREDICT FROM FEATURE STORE GENERATED DATA
test_df = spine_df.limit(3).select("ID")
enriched_df = fs.retrieve_feature_values(
    test_df, training_data.load_features())
enriched_df = enriched_df.drop('ID')
enriched_df.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CASUAL"  |"HOLIDAY"  |"HOUR"  |"MONTH"  |"REGISTERED"  |"WEEKDAY"  |"WORKINGDAY"  |"ATEMP"  |"HUMIDITY"  |"QUARTER"  |"SEASON"  |"TEMP"  |"WEATHER"  |"WINDSPEED"       |"YEAR"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|3         |0          |0       |1        |13            |6          |0             |14.395   |81          |1          |1         |9.84    |1          |12.7993954069447  |2011    |
|8         |0          |1       |1        |32            |6          |0             |13.635   |80          |1          |1         |9.02    |1          |12.7993954069447  |2011    |
|5         |0          |2       |1        |27            |6          |0             |13.635   |

In [ ]:
pred = grid_search.predict(enriched_df)
pred.show()

The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'xgboost' in the local environment is 1.7.6, which does not fit the criteria for the requirement 'xgboost==1.7.3'. Your UDF might not work when the package version is different between the server and your local environment.


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"HOUR"  |"TEMP"  |"WEATHER"  |"CASUAL"  |"WORKINGDAY"  |"SEASON"  |"REGISTERED"  |"WEEKDAY"  |"MONTH"  |"WINDSPEED"       |"HUMIDITY"  |"QUARTER"  |"ATEMP"  |"HOLIDAY"  |"YEAR"  |"PREDICTED_COUNT"   |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0       |9.84    |1          |3         |0             |1         |13            |6          |1        |12.7993954069447  |81          |1          |14.395   |0          |2011    |43.88359743378152   |
|1       |9.02    |1          |8         |0             |1         |32            |6          |1        |12.7993954069447  |80          |1          |13.635   |0          |2011    |32.182484867

### Feature Store - Support Functions

In [ ]:
# FEATURE VIEW  FROM LIBRARY

#SLICE - GET SUBSET OF FEATURES FROM A VIEW
sliced_fv = full_fv.slice(["TEMP"])
sliced_fv

FeatureViewSlice(feature_view_ref=FeatureView(_name=FULL_FEATURES, _entities=[Entity(name=WEATHER, join_keys=['ID'], desc=)], _feature_df=<snowflake.snowpark.dataframe.DataFrame object at 0x122f69840>, _timestamp_col=None, _desc=, _query=SELECT  *  FROM (( SELECT "SEASON" AS "SEASON", "WEATHER" AS "WEATHER", "TEMP" AS "TEMP", "ATEMP" AS "ATEMP", "HUMIDITY" AS "HUMIDITY", "WINDSPEED" AS "WINDSPEED", "QUARTER" AS "QUARTER", "YEAR" AS "YEAR", "ID" AS "ID" FROM (SELECT * FROM SNOWPARK.TUTORIAL.WEATHER_FEATURES$V1)) AS SNOWPARK_LEFT INNER JOIN ( SELECT "HOLIDAY" AS "HOLIDAY", "WORKINGDAY" AS "WORKINGDAY", "CASUAL" AS "CASUAL", "REGISTERED" AS "REGISTERED", "COUNT" AS "COUNT", "HOUR" AS "HOUR", "MONTH" AS "MONTH", "WEEKDAY" AS "WEEKDAY", "ID" AS "ID" FROM (SELECT *  FROM SNOWPARK.TUTORIAL.RENTAL_FEATURES$V1)) AS SNOWPARK_RIGHT USING (ID)), _version=V1, _status=FeatureViewStatus.STATIC, _feature_desc=OrderedDict([('SEASON', None), ('WEATHER', None), ('TEMP', None), ('ATEMP', None), ('HUMIDITY

In [ ]:
#FEATURE VIEW PHYSICAL NAME
print(full_fv.physical_name())

#FEATURE VIEW QUALIFIED NAME
print(full_fv.fully_qualified_name())

FULL_FEATURES$V1
SNOWPARK.TUTORIAL.FULL_FEATURES$V1


In [ ]:
#SET DESCRIPTION TO FEATURE

full_fv = full_fv.attach_feature_desc({"TEMP": "Current Temperature"})

In [ ]:
# FEATURE STORE FROM LIBRARY

#GET FEATURE VIEW
fs.get_feature_view("FULL_FEATURES", "V1").to_df(session).show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"NAME"         |"ENTITIES"             |"TIMESTAMP_COL"  |"DESC"  |"QUERY"                                             |"VERSION"  |"STATUS"                  |"FEATURE_DESC"         |"REFRESH_FREQ"  |"DATABASE"  |"SCHEMA"  |"WAREHOUSE"  |"REFRESH_MODE"  |"REFRESH_MODE_REASON"  |"PHYSICAL_NAME"   |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|FULL_FEATURES  |[                      |NULL             |        |SELECT  *  FROM (( SELECT "SEASO

In [ ]:
#SUSPEND A FEATURE VIEW
fs.suspend_feature_view(full_fv)

#RESUME A FEATURE VIEW
fs.resume_feature_view(full_fv)

fs.get_feature_view("FULL_FEATURES", "V1").to_df(session)["NAME", "STATUS"].show()

--------------------------------------------
|"NAME"         |"STATUS"                  |
--------------------------------------------
|FULL_FEATURES  |FeatureViewStatus.STATIC  |
--------------------------------------------



In [ ]:
# DELETE A FEATURE VIEW

fs.list_feature_views(entity_name="WEATHER").select(["NAME"]).show()
fs.delete_feature_view(fv)

--------------------
|"NAME"            |
--------------------
|RENTAL_FEATURES   |
|WEATHER_FEATURES  |
|FULL_FEATURES     |
--------------------



In [ ]:
#LIST ENTITIES

entity = Entity(name="TEST", join_keys=["ID"])
fs.register_entity(entity)
fs.list_entities().show()

----------------------------------
|"NAME"   |"JOIN_KEYS"  |"DESC"  |
----------------------------------
|TEST     |ID           |        |
|WEATHER  |ID           |        |
----------------------------------



In [ ]:
# GET ENTITY
fs.get_entity("TEST")

Entity(name=TEST, join_keys=['ID'], desc=)

In [ ]:
#DELETE ENTITY
fs.delete_entity("TEST")
fs.list_entities().show()

FeatureStore.delete_entity() is in private preview since 1.0.8. Do not use it in production. 


----------------------------------
|"NAME"   |"JOIN_KEYS"  |"DESC"  |
----------------------------------
|WEATHER  |ID           |        |
----------------------------------



In [ ]:
#load data into feature view from dataset
fs.load_feature_views_from_dataset(training_data)

[FeatureViewSlice(feature_view_ref=FeatureView(_name=FULL_FEATURES, _entities=[Entity(name=WEATHER, join_keys=['ID'], desc=)], _feature_df=<snowflake.snowpark.dataframe.DataFrame object at 0x1381108e0>, _timestamp_col=None, _desc=, _query=SELECT  *  FROM (( SELECT "SEASON" AS "SEASON", "WEATHER" AS "WEATHER", "TEMP" AS "TEMP", "ATEMP" AS "ATEMP", "HUMIDITY" AS "HUMIDITY", "WINDSPEED" AS "WINDSPEED", "QUARTER" AS "QUARTER", "YEAR" AS "YEAR", "ID" AS "ID" FROM (SELECT * FROM SNOWPARK.TUTORIAL.WEATHER_FEATURES$V1)) AS SNOWPARK_LEFT INNER JOIN ( SELECT "HOLIDAY" AS "HOLIDAY", "WORKINGDAY" AS "WORKINGDAY", "CASUAL" AS "CASUAL", "REGISTERED" AS "REGISTERED", "COUNT" AS "COUNT", "HOUR" AS "HOUR", "MONTH" AS "MONTH", "WEEKDAY" AS "WEEKDAY", "ID" AS "ID" FROM (SELECT *  FROM SNOWPARK.TUTORIAL.RENTAL_FEATURES$V1)) AS SNOWPARK_RIGHT USING (ID)), _version=V1, _status=FeatureViewStatus.STATIC, _feature_desc=OrderedDict([('SEASON', None), ('WEATHER', None), ('TEMP', None), ('ATEMP', None), ('HUMIDIT

In [ ]:
#CLEAR ALL THE ITEMS IN THE FEATURE STORE
fs.clear()